In [ ]:
!pip install accelerate -U

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from torch.utils.data import Dataset, DataLoader
import torch

In [ ]:
df = pd.read_csv('path/to/data')

In [ ]:
df = df[['evidence','observation']]

In [ ]:
from sklearn.utils import resample

# Separate classes
proficient_class = df[df['observation'] == 'Proficient']
basic_class = df[df['observation'] == 'Basic']
accomplished_class = df[df['observation'] == 'Accomplished']
unsatisfactory_class = df[df['observation'] == 'Unsatisfactory']

# Determine the size to upsample to (e.g., match the largest class: 'Proficient')
target_size = 5962

# Upsample the minority classes
basic_upsampled = resample(basic_class, replace=True, n_samples=target_size, random_state=42)
accomplished_upsampled = resample(accomplished_class, replace=True, n_samples=target_size, random_state=42)
unsatisfactory_upsampled = resample(unsatisfactory_class, replace=True, n_samples=target_size, random_state=42)

# Combine the upsampled minority classes with the majority class
data_balanced = pd.concat([proficient_class, basic_upsampled, accomplished_upsampled, unsatisfactory_upsampled])

In [ ]:
df = data_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(data_balanced['observation'].value_counts())

# Strip whitespace from labels
df['observation'] = df['observation'].str.strip()

In [ ]:
# Re-check unique values
print(df['observation'].unique())

df['observation'] = df['observation'].str.lower()

label_mapping = {'unsatisfactory': 0, 'basic': 1, 'proficient': 2, 'accomplished': 3}
df['label'] = df['observation'].map(label_mapping)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df['text'] = df['evidence'].apply(lambda x: x.lower())
df['label'] = df['observation']

# Clean the 'observation' column
df['observation'] = df['observation'].str.strip()


In [ ]:
label_mapping = {'unsatisfactory': 0, 'basic': 1, 'proficient': 2, 'accomplished': 3}

df['label'] = df['observation'].map(label_mapping).astype(int)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)


In [ ]:
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=4)

In [ ]:
model.to(device)

# Define label mapping
label_mapping = {'unsatisfactory': 0, 'basic': 1, 'proficient': 2, 'accomplished': 3}

# Map labels to integers
df['label'] = df['observation'].map(label_mapping)

In [ ]:
# Remove rows with unmapped (NaN) values
if df['label'].isna().any():
    print("Removing unmapped values...")
    print("Unmapped values found:", df[df['label'].isna()])
    df = df.dropna(subset=['label'])
    print(f"Dataset size after removal: {df.shape}")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [ ]:
class ObservationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Ensure labels are integers
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ObservationDataset(train_encodings, train_labels.tolist())
test_dataset = ObservationDataset(test_encodings, test_labels.tolist())

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,                # Increase epochs
    per_device_train_batch_size=16,    # Reduce batch size
    per_device_eval_batch_size=32,     # Adjust eval batch size
    warmup_steps=200,                  # Reduce warmup steps
    weight_decay=0.1,                  # Increase weight decay
    learning_rate=2e-5,                # Lower learning rate
    logging_dir='./logs',
    logging_steps=5,                   # More frequent logging
    evaluation_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=2,     # Simulate larger batch size
    fp16=True if device.type == 'cuda' else False,
    load_best_model_at_end=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

In [ ]:
predictions = trainer.predict(test_dataset)

pred_logits = predictions.predictions  # Raw logits
pred_labels = pred_logits.argmax(axis=1)  # Predicted class indices

true_labels = predictions.label_ids

print("True Labels:", true_labels[:10])
print("Predicted Labels:", pred_labels[:10])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(true_labels, pred_labels)
print("Confusion Matrix:\n", cm)

label_mapping = {0: "Unsatisfactory", 1: "Basic", 2: "Proficient", 3: "Accomplished"}
report = classification_report(true_labels, pred_labels, target_names=label_mapping.values())
print("Classification Report:\n", report)


In [ ]:
trainer.save_model('./trained_model')

# Save the tokenizer
tokenizer.save_pretrained('./trained_model')


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('./trained_model')
tokenizer = DistilBertTokenizer.from_pretrained('./trained_model')

In [ ]:
text = "They were terrific, everybody loved them, accomplished the task well."

# Tokenize the input
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

# Get predictions
model.eval()  # Set model to evaluation mode
outputs = model(**inputs)

# Predicted class
predicted_class = torch.argmax(outputs.logits, dim=1).item()
print(f"Predicted class: {predicted_class}")

In [ ]:
!zip -r /content/trained_model.zip /content/trained_model

from google.colab import files
files.download('/content/trained_model.zip')